In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import math
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as wg
from scipy.signal import find_peaks
import sys
sys.path.append("../src")
import importlib
import support
import visualizations
import calibrations
import kinematics
from scipy import signal
importlib.reload(support)
from support import readFileDialog, parse_data_file, getFrame, writeToDATA, saveFileDialog
importlib.reload(kinematics)
from kinematics import getKeypointsCoord, getKeypointsPixels, inverseKinematicsRowing
importlib.reload(visualizations)
from visualizations import showFrame, keypointsDATAtoFrame
importlib.reload(calibrations)
from calibrations import getMmppInterface, defineReference
%matplotlib widget

# Functions 

In [2]:
def readFileVicon(file_path, prefix="Victor:", keypoints_mapping_Vicon=None, first_line="Trajectories"):
    if keypoints_mapping_Vicon == None:
        keypoints_mapping_Vicon = ["LANK", "LKNE", "LTHI", "LASI", "LSHO", "LELB", "LWRB", "RANK", "RKNE", "RTHI", "RASI", "RSHO", "RELB", "RWRB"]
    metadata = pd.read_csv(file_path, nrows=1)
    fps = pd.read_csv(file_path, nrows=1)[first_line][0]
    keypoints_mapping_Vicon_tmp = pd.read_csv(file_path, skiprows=[0,1], nrows=1, header=None).values.tolist()[0]
    vicon_data = np.array(pd.read_csv(file_path, skiprows=[0,1,2,4]))
    keypoints_Vicon = np.zeros([len(vicon_data), len(keypoints_mapping_Vicon), 3])
    for i in range(len(keypoints_mapping_Vicon)):
        idx = keypoints_mapping_Vicon_tmp.index(prefix + keypoints_mapping_Vicon[i])
        keypoints_Vicon[:, i, :] = vicon_data[:, idx:idx+3]
    return keypoints_Vicon, keypoints_mapping_Vicon, fps

In [3]:
def plot2DPose(keypoints_Vicon):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(keypoints_Vicon[:, 0], keypoints_Vicon[:, 1], marker='o', color='r')
    for i in range(keypoints_Vicon.shape[0]-1):
        ax.plot([keypoints_Vicon[i, 0], keypoints_Vicon[i+1, 0]], [keypoints_Vicon[i, 1], keypoints_Vicon[i+1, 1]], color='black')
    ax.set_xlabel('X Axis')
    ax.set_ylabel('Y Axis')
    # Fix aspect ratio
    x, y = keypoints_Vicon[:, 0], keypoints_Vicon[:, 1]
    max_range = np.array([x.max()-x.min(), y.max()-y.min()]).max() / 2.0
    mean_x = x.mean()
    mean_y = y.mean()
    max_range += 400
    ax.set_xlim(mean_x - max_range, mean_x + max_range)
    ax.set_ylim(mean_y - max_range, mean_y + max_range)
    plt.grid()
    plt.show()

In [4]:
def get_line_function_from_points(x1, y1, x2, y2):
    corr = 0
    if (x2 == x1):
        corr = 1e-8
    m = (y2-y1)/(abs(x2-x1+corr))
    b = y1 - m*x1
    return m, b

In [5]:
def get_intersection_from_lines(m1, b1, m2, b2):
    corr = 0
    if (m2 == m1):
        corr = 1e-8
    x = (b2-b1)/(abs(m1-m2+corr))
    y = (b1*m2 - b2*m1)/(m2 - m1)
    return x, y

In [44]:
def correct_knee_joint(kp, kp_mapping):
    kp_RASI = kp[kp_mapping.index('RASI'), :]
    x1_1, y1_1 = kp_RASI[0], kp_RASI[1]
    kp_RTHI = kp[kp_mapping.index('RTHI'), :]
    x2_1, y2_1 = kp_RTHI[0], kp_RTHI[1]
    kp_RKNE = kp[kp_mapping.index('RKNE'), :]
    x1_2, y1_2 = kp_RKNE[0], kp_RKNE[1]
    kp_RANK = kp[kp_mapping.index('RANK'), :]
    x2_2, y2_2 = kp_RANK[0], kp_RANK[1]
    m1, b1 = get_line_function_from_points(x1_1, y1_1, x2_1, y2_1)
    m2, b2 = get_line_function_from_points(x1_2, y1_2, x2_2, y2_2)
    kp_out = np.zeros([kp.shape[0]-1, kp.shape[1]])
    kp_out[:1, :] = kp[:1, :]
    kp_out[2:, :] = kp[3:, :]

    x_knee, y_knee = get_intersection_from_lines(m1, b1, m2, b2)
    if (x_knee < x2_1) or (y_knee < y1_2):
        x_knee = (x2_1 + x1_2)/2
        y_knee = (y2_1 + y1_2)/2
    kp_out[1, :] = [x_knee, y_knee]
    
    kp_mapping = ['Right Ankle',
                  'Right Knee',
                  'Right Hip',
                  'Right Shoulder',
                  'Right Elbow',
                  'Right Wrist']
    return kp_out, kp_mapping

In [7]:
def get2DAnglesVector(keypoints_Vicon_2D, keypoints_mapping_Vicon=None):
    if keypoints_mapping_Vicon == None:
            keypoints_mapping_Vicon = ["ANK", "KNE", "HIP", "SHO", "ELB", "WRI"]
    angles_vec = np.zeros([len(keypoints_Vicon), len(keypoints_mapping_Vicon)-1])
    for i in range(len(keypoints_Vicon)):
        angles = inverseKinematicsRowing(keypoints_Vicon_2D[i])
        angles_vec[i, :] = angles
    return angles_vec

# Read Vicon file

In [8]:
file_path_vicon = readFileDialog("Open Vicon file")
file_path_vicon

'C:/Users/victo/Documents/Motion_analysis/Vicon/Celio_Vicon/Voga24_5min_Celio.csv'

In [9]:
keypoints_Vicon, keypoints_mapping_Vicon, fps_vicon = readFileVicon(file_path_vicon, prefix="Celio:")

In [45]:
kp_right = keypoints_Vicon[:, int(keypoints_Vicon.shape[1]/2):]
kp_right_mapping = keypoints_mapping_Vicon[int(len(keypoints_mapping_Vicon)/2):]
kp_Vicon_2D = kp_right[:, :, 1:]
kp_Vicon_2D_corrected = np.zeros([kp_Vicon_2D.shape[0], kp_Vicon_2D.shape[1]-1, kp_Vicon_2D.shape[2]])
for i in range(len(kp_Vicon_2D_corrected)):
    kp_Vicon_2D_corrected[i, :, :], kp_mapping = correct_knee_joint(kp_Vicon_2D[i, :, :], kp_right_mapping)

In [17]:
file_path_op = readFileDialog("Open Openpose file")
file_path_op

'C:/Users/victo/Documents/Motion_analysis/Vicon/Celio_Vicon/Voga24_5min_Celio.data'

In [46]:
data, var_names = parse_data_file(file_path_op)
kp_openpose = data["keypoints"]
md_openpose = data["metadata"]
ang_openpose = data["angles"]
fps_openpose = md_openpose["fps"]
angles_names = md_openpose['angles_names']

In [47]:
md_vicon = md_openpose.copy()

In [48]:
md_vicon["fps"] = 100
md_vicon["n_frames"] = len(kp_Vicon_2D_corrected)
t_vicon = np.linspace(0, md_vicon["n_frames"]*(1/md_vicon['fps']), md_vicon["n_frames"])

In [49]:
angles_vec_2D_corrected = get2DAnglesVector(kp_Vicon_2D_corrected)
angles_vec_2D = get2DAnglesVector(kp_Vicon_2D)

In [53]:
output_path = saveFileDialog()

In [54]:
output_path

'C:/Users/victo/Documents/Motion_analysis/Vicon/Celio_Vicon/Voga24_5min_Celio.data'

In [55]:
writeToDATA(output_path, md_vicon, write_mode='w')
for i in range(len(kp_Vicon_2D_corrected)):
    file_data = {'keypoints': kp_Vicon_2D_corrected[i].tolist(),
                 'angles': angles_vec_2D_corrected[i].tolist()}
    writeToDATA(output_path, file_data, write_mode='a')

In [51]:
joint_angle = wg.Dropdown(options=angles_names,
                        description='Angle:',
                        value='Shoulder <- Hip -> Knee',
                        disabled=False)

def plot_2d_comparison(joint_angle):
    angle_idx = angles_names.index(joint_angle)
    plt.close('all')
    plt.figure()
    plt.plot(t_vicon, angles_vec_2D[:, angle_idx], label='Captured')
    plt.plot(t_vicon, angles_vec_2D_corrected[:, angle_idx],  label='Corrected')
    angle_name = joint_angle.split('->')[0].split('<-')[-1].replace(" ", "")
    plt.title("2D Comparation for the {}".format(angle_name))
    plt.xlabel("Time (s)")
    plt.ylabel("Angle (Degrees)")
    plt.grid(True)
    plt.legend()
    plt.show()
    
out = wg.interactive_output(plot_2d_comparison, {'joint_angle': joint_angle})

vbox = wg.VBox([joint_angle, out])
display(vbox)

In [52]:
t_sec = 6.5
n_frame = int(round(t_sec * md_vicon["fps"]))
kp = kp_Vicon_2D[n_frame,:,:]
kp_corr = kp_Vicon_2D_corrected[n_frame,:,:]
plt.close('all')
plot2DPose(kp)
plot2DPose(kp_corr)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …